In [3]:
import requests
from bs4 import BeautifulSoup
from urllib import request
from urllib.request import Request, urlopen
agent = {"User-Agent":'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}

In [4]:
#define new category aside from the home page
category = ['iindaba','ezemidlalo','ezoyolo','izimvo']
#get the number of `featured` pages each of the categories have
def get_no_featuredpages(catgory):
  url = "https://www.isolezwelesixhosa.co.za/"+catgory+"?filter_by=featured"
  reqs = requests.get(url, headers=agent)
  soup = BeautifulSoup(reqs.content, 'lxml')
  res = soup.find('span',{'class':'pages'})
  return int(res.text.split()[-1])

#scrap links from the different categories 
def scrap_link(catgory, catgory_size):
  urls = []
  print("Getting the URL from the category : ", catgory, " (featured) with ", catgory_size, " pages . " )
  for p in range(catgory_size):
    url = 'https://www.isolezwelesixhosa.co.za/'+catgory+'/page/'+str(p+1)+'?filter_by=featured'
    #print(url)
    reqs = requests.get(url, headers=agent)
    soup = BeautifulSoup(reqs.content, 'lxml')
    res = soup.find_all('div',{'class':'td-module-thumb'})#.find_all("a", href=True)
    #print(res)
    urls.extend([t.a['href'] for t in res])
  #return all the news article links from the category
  return urls


In [5]:
#scrap the articles with links in the `urls` variable
def getSoup(links):
  print("getting soup objects for all the links ... ")
  soupx = []
  cnt=0
  for url in links: 
    page_request = request.Request(url, headers=agent)
    page = request.urlopen(page_request)
    #response = requests.get(page_url)
    #print(page.getcode())
    soup = BeautifulSoup(page, 'html.parser')
    soupx.append(soup)
    cnt = cnt + 1
    progresse = (cnt/len(links) * 100) 
    if progresse  % 10 == 0:
      print ("Got ", progresse, "% of soup objects")
  print("got the soup object for all the links ... ")
  return soupx

def getcontent(soupx):
  print("scrapping the articles ... ")
  cnt=0;
  title=[];time=[]; texts=[]
  for soups in soupx:
    cnt=cnt+1
    if soups.find("h1", {"class":"entry-title"})!= None:
      title.append(soups.find("h1", {"class":"entry-title"}).text.replace("\t","").replace("\n",""))
    else:
      title.append("")
    if soups.find("time", {"class":"entry-date updated td-module-date"})!= None:
      time.append(soups.find("time", {"class":"entry-date updated td-module-date"}).text.replace("\t","").replace("\n",""))
    else:
      time.append("")
  
    if soups.find("div", {"class":"td-post-content"})!=None:
      result = soups.find("div", {"class":"td-post-content"}).findAll('p')
      txtstring=""
      for x in result:
      #print (x.text)
        txtstring+=x.text.replace(u'\xa0', u' ').replace('\n'," ")+" \n"
      texts.append(txtstring.strip())
    else:
      texts.append("") 

    progresse = (cnt/len(soupx) * 100) 
    if progresse  % 10 == 0:
      print ("Scrapped ", progresse, "% of the articles")

  return title,time,texts

In [6]:
cat_len = [get_no_featuredpages(cat) for cat in category]
print("Got the following size for each category = ", cat_len)
urls = set([scrap_link(category[i], cat_len[i]) for i in range(len(category))][0])

Got the following size for each category =  [916, 386, 39, 4]
Getting the URL from the category :  ezoyolo  (featured) with  39  pages . 
Getting the URL from the category :  izimvo  (featured) with  4  pages . 


In [8]:
soups = getSoup(list(urls))
title,time,texts = getcontent(soups)
#create a dictionarty
import pandas as pd
d = {'Date':time,'Title':title,'Text':texts}
df = pd.DataFrame(d)

getting soup objects for all the links ... 
Got  50.0 % of soup objects
Got  100.0 % of soup objects
got the soup object for all the links ... 
Scrapped  50.0 % of the articles
Scrapped  100.0 % of the articles


In [18]:
print(df)

,Date,Title,Text
0,"Jul 20, 2016",Abadlali abane bayayishiya i-7de Laan!,ABADLALI bomdlalo we-Afrikaans kwiSABC 2 i-7de...
1,"Apr 18, 2016",Kumabonakude wakho kule veki,Isidingo \nULincoln uyama ngoSkhumbuzo ukuze a...
2,"Apr 2, 2020",Kumabonakude wakho kule veki,UThulani noGodfather bayaxabana. \nUSbu unesi...
3,"Apr 12, 2016",Uzibeka phantsi umdlali we-7de Laan!,KUPHINDE kwabhengezwa ukuba kukho omnye oyishi...
4,"Jul 5, 2016",Bonwabele ukondliwa yibhrakfesi kaDJ Sbu!,Ngokwenene isidlo seBhrakfesi sisidlo sokuqala...
...,...,...,...
389,"Nov 9, 2015",Kumabonakude wakho kule veki,Rhythm City \nUThemba wenza konke okusemandlen...
390,"Apr 15, 2016",Isolezwe Gig Guideno-Agent Lungaz,"Lwesihlanu, Tshazimpuzi 15 \n* Wise Guys (Vi..."
391,"Jan 25, 2016",Ubumenye-menye beekhamera kwilokishi yaseDimb...,ISIKHULULO sikamabonakude iBay TV sifak’isandl...
392,"Apr 26, 2018",UTete ugxeleshe itumente enemali entle,"Umakhwekhwetha wembethimanqindi yaseMdantsane,..."


In [19]:
#write the dataframe to file
df.to_csv(r'xhosa_news.csv')